In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
print('Available Stock ID: \n',np.unique(train['stock_id']))

In [ ]:
def f1(i1,i2):
    StockId = i1
    book_example = []
    trade_example = []
    train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
    if int(StockId) in np.unique(train['stock_id']).tolist():
        train1 = train[train['stock_id']==int(StockId)]
        StockID_S = str(StockId)
        book_example = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=' + StockID_S)
        trade_example =  pd.read_parquet('../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=' + StockID_S)
        TimeId = i2
        TimeID_S = str(TimeId)
        book_example = book_example[book_example['time_id']==int(TimeId)]
        if book_example.empty:
            print('\nTimeID is not available')
            print('\nAvailable TimeId\n\n', np.unique(trade_example['time_id']))
        trade_example = trade_example[trade_example['time_id']==int(TimeId)]
        book_example.loc[:,'stock_id'] = StockId
        trade_example.loc[:,'stock_id'] = StockId
        book_example['wap'] = (book_example['bid_price1'] * book_example['ask_size1'] +
                                    book_example['ask_price1'] * book_example['bid_size1']) / (
                                           book_example['bid_size1']+ book_example['ask_size1'])
        book_example.loc[:,'log_return'] = np.log((book_example['wap'])).diff()
        fig = px.line(book_example, x="seconds_in_bucket", y="wap", title='WAP of stock_id_' + StockID_S + ', time_id_' + TimeID_S)
        fig.show()
        book_example1 = book_example[~book_example['log_return'].isnull()]
        fig = px.line(book_example, x="seconds_in_bucket", y="log_return", title='log_return of stock_id_' + StockID_S + ', time_id_' + TimeID_S)
        fig.show()
        realized_volatility = np.sqrt(np.sum(book_example['log_return']**2))
        print('realized_volatility for stock_id ' + StockID_S + ' and time_id ' + TimeID_S + ' is:', realized_volatility)
        target = float(train1[train1['time_id']==int(TimeId)]['target'])
        print('target for stock_id ' + StockID_S + ' and time_id ' + TimeID_S + ' is:', target)
    else:
        print('\nStockID is not available')
        print('\nAvailable StockId\n\n', np.unique(train['stock_id']))
    return book_example, trade_example

In [ ]:
i1 = 0          #Enter StockID
i2 = 5          #Enter TimeID
book_example, trade_example = f1(i1,i2)
print('\n\n Book_data \n\n ',book_example.head())
print('\n\n',book_example.tail())
print('\n\n Trade_data \n\n ',trade_example.head())
print('\n\n',trade_example.tail())